# Projeto WPT - Grupo 2
Arthur Tamm, Caio Bôa, Gabriel Hermida e Pedro Civita

## Importando Bibliotecas

In [9]:
import numpy as np
from capacitor import Capacitor
from indutor import Indutor
from utils import *

# Etapa 1

In [10]:
l1  = 10e-6
c1 = 100e-6

# f = freqResonancia(l1, c1)
# f

5032.921210448703

In [11]:
frequencies = np.arange(1000, 200000, 100)

l_list = [10e-6, 20e-6, 30e-6, 40e-6, 50e-6]



